In [1]:
x=1

In [2]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
#import multigrate as mtg
#import multimil as mtm
import scanpy as sc
import ast
import anndata as ad
import yaml
from pprint import pprint

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('../../pipeline/data/reports/methods.tsv', sep='\t', index_col=0)
df

,precision,recall,f1-score,support,split,method,epoch,hash,method_params,task,query_epoch
Unnamed: 0,,,,,,,,,,,
guided,1.000000,0.566264,0.723076,98862.000000,0,multigrate_mil,19.0,00e4d1cd7b,"{'lr': 0.001, 'rna_indices_end': 30, 'categori...",organoid_atlas_subset,NaN
unguided,0.008532,1.000000,0.016920,369.000000,0,multigrate_mil,19.0,00e4d1cd7b,"{'lr': 0.001, 'rna_indices_end': 30, 'categori...",organoid_atlas_subset,NaN
accuracy,0.567877,0.567877,0.567877,0.567877,0,multigrate_mil,19.0,00e4d1cd7b,"{'lr': 0.001, 'rna_indices_end': 30, 'categori...",organoid_atlas_subset,NaN
macro avg,0.504266,0.783132,0.369998,99231.000000,0,multigrate_mil,19.0,00e4d1cd7b,"{'lr': 0.001, 'rna_indices_end': 30, 'categori...",organoid_atlas_subset,NaN
weighted avg,0.996313,0.567877,0.720450,99231.000000,0,multigrate_mil,19.0,00e4d1cd7b,"{'lr': 0.001, 'rna_indices_end': 30, 'categori...",organoid_atlas_subset,NaN
...,...,...,...,...,...,...,...,...,...,...,...
weighted avg,0.833333,0.500000,0.500000,4.000000,3,pb_nn,NaN,fe30155720,"{'norm': 0, 'epochs': 200, 'lr': 0.0001, 'batc...",organoid_atlas_subset,NaN
1,1.000000,1.000000,1.000000,4.000000,4,pb_nn,NaN,fe30155720,"{'norm': 0, 'epochs': 200, 'lr': 0.0001, 'batc...",organoid_atlas_subset,NaN
accuracy,1.000000,1.000000,1.000000,1.000000,4,pb_nn,NaN,fe30155720,"{'norm': 0, 'epochs': 200, 'lr': 0.0001, 'batc...",organoid_atlas_subset,NaN


In [6]:
config = {
    'TASKS': ['organoid_atlas_subset'],
}

In [12]:
best_methods = {}

for task in config['TASKS']:
    n_splits = 5 #config['TASKS'][task]['n_splits']
    df_task = df[df['task'] == task]
    best_methods_per_task = {}

    for method in np.unique(df_task['method']):
        df_method = df_task[df_task['method'] == method]
        best_accuracy = -1
        best_hash = None
        best_accuracies = None

        for h in np.unique(df_method['hash']):
            df_tmp = df_method[df_method['hash'] == h]
            # for multigrate need to check if all epochs and query_epochs are present and group by those
            # check only if all splits are present
            if method == 'multigrate_mil':
                if df_tmp.drop(['epoch', 'query_epoch']).isnull().values.any():
                    continue
                for epoch in np.unique(df_tmp['epoch']):
                    df_tmp_epoch = df_tmp[df_tmp['epoch'] == epoch]
                    if len(np.unique(df_tmp_epoch['split'])) != n_splits:
                        continue
                    accuracies = []
                    for i in range(n_splits):
                        accuracies.append(df_tmp_epoch[df_tmp_epoch['split'] == i]['f1-score']['accuracy'])
                        accuracy = np.mean(accuracies)
                        if accuracy > best_accuracy:
                            best_accuracy = accuracy
                            best_hash = h
                            best_params = df_tmp_epoch['method_params'].iloc[0]
                            best_epoch = epoch
                            best_query_epoch = np.nan
                            best_accuracies = accuracies
            elif method == 'multigrate':
                if df_tmp.isnull().values.any():
                    continue
                for epoch in np.unique(df_tmp['epoch']):
                    df_tmp_epoch = df_tmp[df_tmp['epoch'] == epoch]
                    if len(np.unique(df_tmp_epoch['split'])) != n_splits:
                        continue
                    for query_epoch in np.unique(df_tmp_epoch['query_epoch']):
                        df_tmp_q_epoch = df_tmp_epoch[df_tmp_epoch['query_epoch'] == query_epoch]
                        if len(np.unique(df_tmp_q_epoch['split'])) != n_splits:
                            continue
                        accuracies = []
                        for i in range(n_splits):
                            accuracies.append(df_tmp_q_epoch[df_tmp_q_epoch['split'] == i]['f1-score']['accuracy'])
                            accuracy = np.mean(accuracies)
                            if accuracy > best_accuracy:
                                best_accuracy = accuracy
                                best_hash = h
                                best_params = df_tmp_q_epoch['method_params'].iloc[0]
                                best_epoch = epoch
                                best_query_epoch = query_epoch
                                best_accuracies = accuracies

            else:
                if len(np.unique(df_tmp['split'])) != n_splits:
                    continue
                accuracies = []
                for i in range(n_splits):
                    accuracies.append(df_tmp[df_tmp['split'] == i]['f1-score']['accuracy'])
                accuracy = np.mean(accuracies)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_hash = h
                    best_params = df_tmp['method_params'].iloc[0]
                    best_epoch = np.nan
                    best_query_epoch = np.nan
                    best_accuracies = accuracies

        if best_hash is not None: # this can happen if e.g. all multigrate jobs failed because of a too high lr
            best_methods_per_task[method] = {
                'hash': best_hash,
                'accuracy': best_accuracy,
                'best_params': best_params,
                'best_epoch': best_epoch,
                'best_query_epoch': best_query_epoch,
                'accuracies': best_accuracies,
                'method': method,
            }
    best_df = pd.DataFrame.from_dict(best_methods_per_task).T
    best_df['task'] = task
    best_methods[task] = best_df

best_all = pd.concat(best_methods)


here


KeyError: "['epoch' 'query_epoch'] not found in axis"

In [13]:
best_all

hash  accuracy  \
organoid_atlas_subset gex_nn  86b7d51ea3  0.521979   
                      gex_rf  c2b168fca4  0.592501   
                      pb_nn   626fef9794      0.69   
                      pb_rf   7285549255      0.57   

                                                                    best_params  \
organoid_atlas_subset gex_nn  {'norm': 0, 'epochs': 30, 'lr': 0.0001, 'batch...   
                      gex_rf                                        {'norm': 0}   
                      pb_nn   {'norm': 0, 'epochs': 200, 'lr': 0.0001, 'batc...   
                      pb_rf                                         {'norm': 0}   

                             best_epoch best_query_epoch  \
organoid_atlas_subset gex_nn        NaN              NaN   
                      gex_rf        NaN              NaN   
                      pb_nn         NaN              NaN   
                      pb_rf         NaN              NaN   

                                                                     accuracies  \
organoid_atlas_subset gex_nn  [0.99611008656569, 0.4518193514333784, 0.11393...   
                      gex_rf  [0.4208261531174733, 0.8730577639956454, 0.398...   
                      pb_nn                          [0.2, 1.0, 0.5, 0.75, 1.0]   
                      pb_rf                          [0.6, 1.0, 0.5, 0.25, 0.5]   

                              method                   task  
organoid_atlas_subset gex_nn  gex_nn  organoid_atlas_subset  
                      gex_rf  gex_rf  organoid_atlas_subset  
                      pb_nn    pb_nn  organoid_atlas_subset  
                      pb_rf    pb_rf  organoid_atlas_subset